In [14]:
import math
#variables

boltDia = 0.625
shearStrength = 68 #this is the value for an A325 bolt with threads not in the shear plane
boltArea = (boltDia/2)**2/math.pi
numBolts = 3
boltSpacing = 3
cg_x = 4  #this is the distance from the primary beam face to the CG of the bolt group

phi=1 #this is the safety factor for bolt shear for the LRFD method
R_ult = boltArea*shearStrength # this is the nominal shear strength of the bolt
R_nom = phi * boltArea* shearStrength

deltaMax = 0.34 #maximum bolt deformation per AISC 7-13

#determine location of bolts
bgHeight = boltSpacing * (numBolts-1)

boltCord = [(0,bgHeight/2)] #starting from the top bolt

x = 1 #we already did the top bolt
while x < numBolts:
    boltTuple = (0,bgHeight/2-boltSpacing*x)
    boltCord.append(boltTuple)
    x+=1

#initial Ix and Iy assume the IC is at the vertical center of the bolt group (y=0)
icx = 0.6

iterations = 100 #number of times trying to find ic
iteration = 0
while iteration<iterations:
    ic = (icx,0)
    #get a list of distances from each bolt to the IC
    distList = []
    for point in boltCord:
        distList.append(math.dist(ic,point))
    
    maxDist = max(distList)
    
    deltaList = []
    for dist in distList:
        deltaList.append(0.34*(dist/maxDist))
    
    rList = []
    for delta in deltaList:
        rList.append(R_ult*math.pow(1-math.pow(math.e,-10*delta),0.55))
    
    x = 0
    rxList = []
    ryList = []
    mList = [] #list of moments around IC
    for R in rList:
        boltPoint = boltCord[x]
        cx = boltPoint[0]-ic[0]
        cy = boltPoint[1]-ic[1]
        c = distList[x]
        rx = -R*(cy/c)
        ry = R*(cx/c)
        m = -rx*cy+ry*cx
        rxList.append(rx)
        ryList.append(ry)
        mList.append(m)
        x+=1
    
    i=0
    rxSum=0
    rySum=0
    mSum=0
    while i<numBolts:
        rxSum+=rxList[i]
        rySum+=ryList[i]
        mSum+=mList[i]
        i+=1
    
    mSum = round(mSum,2)
    print(f"the sum of bolt moments: {mSum}")
    ex = cg_x + ic[0] #distance from the IC to the applied load
    m_ult = -rySum*ex
    m_ult = round(m_ult,2)
    print(f"the moment of the resultant force around the IC: {m_ult}")
    m_diff = mSum-m_ult
    print(f"IC location: {ic}")
    if m_ult == mSum:
        print("solution found")
        iteration = iterations
    elif mSum>m_ult:
        icx+=m_diff*0.1
    else:
        icx-=m_diff*0.1

    iteration+=1

print(f"resultant reaction force: {-rySum}")
print(f"sum of rlist: {sum(rList)}")
print(f"nominal group strenth: {R_nom}")
r2List = []
for delta in deltaList:
     r2List.append(R_ult*numBolts*math.pow(1-math.pow(math.e,-10*delta),0.55))
print(r2List)


the sum of bolt moments: 14.35
the moment of the resultant force around the IC: 10.89
IC location: (0.6, 0)
the sum of bolt moments: 15.48
the moment of the resultant force around the IC: 15.2
IC location: (0.946, 0)
the sum of bolt moments: 15.58
the moment of the resultant force around the IC: 15.53
IC location: (0.9740000000000001, 0)
the sum of bolt moments: 15.6
the moment of the resultant force around the IC: 15.59
IC location: (0.9790000000000002, 0)
the sum of bolt moments: 15.6
the moment of the resultant force around the IC: 15.6
IC location: (0.9800000000000002, 0)
solution found
resultant reaction force: 3.133318738755393
sum of rlist: 6.162515807462447
nominal group strenth: 2.2381163872297782
[6.590164602161738, 5.3072182180638645, 6.590164602161738]
